In [1]:
# 必要ライブラリのインポート
import numpy as np
import pandas as pd
import os
import pickle
import gc

# 可視化
import matplotlib.pyplot as plt

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

# バリデーション
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# 評価指標
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# モデリング: lightgbm
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

# matplotilbで日本語表示したい場合はこれをinstallしてインポートする
!pip install japanize-matplotlib
import japanize_matplotlib

# 2022/06/02追加: Kaggle notebook環境変更のため
!pip install -U pandas_profiling

# 分布確認
import pandas_profiling as pdp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120275 sha256=55c148785d175c4dd2ee3835a181576cd6be838b8a9249101bff4870cef3b5f9
  Stored in directory: /root/.cache/pip/wheels/d3/7d/c5/d3e02382561888f86edabf3256c09b3298f8e24456f8fc4da3
Successfully built japanize-matplotlib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.9/345.9 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pandas_profiling
    Found existing installation: pandas-profiling 3.6.2
    Uninstalling pandas-profiling-3.6.2:
      Successfully uninstalled pandas-profiling-3.6.2


In [2]:
#ファイル読み込み
df_train = pd.read_csv("../input/titanic/train.csv")
df_train.head()

#Excelファイル　df = pd.read_excel("ファイル名")
#タブ区切り　　　df = pd_read_CSV("sample.tst", sep="\t")
#Shift JIS形式のCSVファイル　df = pd.read_CSV("sample.csv", encoding="Shift_jis")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#データセット作成
x_train, y_train, id_train = df_train[["Pclass", "Fare"]], \
                             df_train[["Survived"]], \
                             df_train[["PassengerId"]]
print(x_train.shape, y_train.shape, id_train.shape)

(891, 2) (891, 1) (891, 1)


In [4]:
#ベイズ最適化
import optuna

In [5]:
#目的関数の定義
# 探索しないハイパーパラメータ
params_base = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.02,
    'n_estimators': 100000,
    "bagging_freq": 1,
    "seed": 123,
}

def objective(trial):
    # 探索するハイパーパラメータ
    params_tuning = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 256),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 200),
        "min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e2, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e2, log=True),
    }
    params_tuning.update(params_base)
    
    # モデル学習・評価
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    for nfold in np.arange(5):
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train.loc[idx_tr, :]
        x_va, y_va = x_train.loc[idx_va, :], y_train.loc[idx_va, :]
        model = lgb.LGBMClassifier(**params_tuning)
        model.fit(x_tr,
                  y_tr,
                  eval_set=[(x_tr,y_tr), (x_va,y_va)],
                  early_stopping_rounds=100,
                  verbose=0,
                 )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = accuracy_score(y_va, np.where(y_va_pred>=0.5, 1, 0))
        list_metrics.append(metric_va)
    
    # 評価値の計算
    metrics = np.mean(list_metrics)
    
    return metrics

In [6]:
#最適化処理（探査の実行）
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction="maximize")
study.optimize(objective, n_trials=30)

[I 2023-04-03 15:00:19,381] A new study created in memory with name: no-name-d7af92f8-fb75-41a2-b50d-004fd4f44100


[LightGBM] [Warning] lambda_l1 is set=0.492522233779106, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.492522233779106
[LightGBM] [Warning] bagging_fraction is set=0.8597344848927815, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8597344848927815
[LightGBM] [Warning] lambda_l2 is set=83.76388146302445, reg_lambda=0.0 will be ignored. Current value: lambda_l2=83.76388146302445
[LightGBM] [Warning] feature_fraction is set=0.7756573845414456, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7756573845414456
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=4.792414358623587e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=4.792414358623587e-05
[LightGBM] [Warning] min_data_in_leaf is set=61, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=61
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1

[I 2023-04-03 15:00:22,056] Trial 0 finished with value: 0.664478061640826 and parameters: {'num_leaves': 181, 'min_data_in_leaf': 61, 'min_sum_hessian_in_leaf': 4.792414358623587e-05, 'feature_fraction': 0.7756573845414456, 'bagging_fraction': 0.8597344848927815, 'lambda_l1': 0.492522233779106, 'lambda_l2': 83.76388146302445}. Best is trial 0 with value: 0.664478061640826.


[LightGBM] [Warning] lambda_l1 is set=0.567922374174008, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.567922374174008
[LightGBM] [Warning] bagging_fraction is set=0.8645248536920208, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8645248536920208
[LightGBM] [Warning] lambda_l2 is set=0.01732652966363563, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01732652966363563
[LightGBM] [Warning] feature_fraction is set=0.6715890080754348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6715890080754348
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.00015009027543233888, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.00015009027543233888
[LightGBM] [Warning] min_data_in_leaf is set=99, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=99
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lam

[I 2023-04-03 15:00:23,465] Trial 1 finished with value: 0.6712196346745339 and parameters: {'num_leaves': 178, 'min_data_in_leaf': 99, 'min_sum_hessian_in_leaf': 0.00015009027543233888, 'feature_fraction': 0.6715890080754348, 'bagging_fraction': 0.8645248536920208, 'lambda_l1': 0.567922374174008, 'lambda_l2': 0.01732652966363563}. Best is trial 1 with value: 0.6712196346745339.


...

[I 2023-04-03 15:01:27,561] Trial 27 finished with value: 0.6824555897307136 and parameters: {'num_leaves': 253, 'min_data_in_leaf': 7, 'min_sum_hessian_in_leaf': 0.0015110604476009741, 'feature_fraction': 0.7981086348840757, 'bagging_fraction': 0.7775152269204683, 'lambda_l1': 0.13213697905758195, 'lambda_l2': 0.03981834857329031}. Best is trial 7 with value: 0.6992530286862093.


[LightGBM] [Warning] lambda_l1 is set=0.05156762599756539, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.05156762599756539
[LightGBM] [Warning] bagging_fraction is set=0.7317397959755858, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7317397959755858
[LightGBM] [Warning] lambda_l2 is set=0.20885772437641695, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.20885772437641695
[LightGBM] [Warning] feature_fraction is set=0.8426315000433712, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8426315000433712
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0068112646256021986, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0068112646256021986
[LightGBM] [Warning] min_data_in_leaf is set=49, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=49
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] l

[I 2023-04-03 15:01:30,377] Trial 28 finished with value: 0.7048207896553889 and parameters: {'num_leaves': 233, 'min_data_in_leaf': 49, 'min_sum_hessian_in_leaf': 0.0068112646256021986, 'feature_fraction': 0.8426315000433712, 'bagging_fraction': 0.7317397959755858, 'lambda_l1': 0.05156762599756539, 'lambda_l2': 0.20885772437641695}. Best is trial 28 with value: 0.7048207896553889.


[LightGBM] [Warning] lambda_l1 is set=0.0223517282592696, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0223517282592696
[LightGBM] [Warning] bagging_fraction is set=0.7326320350377127, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7326320350377127
[LightGBM] [Warning] lambda_l2 is set=0.265177341262348, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.265177341262348
[LightGBM] [Warning] feature_fraction is set=0.8381244831959944, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8381244831959944
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.006296757603803257, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.006296757603803257
[LightGBM] [Warning] min_data_in_leaf is set=57, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=57
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1

[I 2023-04-03 15:01:32,165] Trial 29 finished with value: 0.6733915008474044 and parameters: {'num_leaves': 161, 'min_data_in_leaf': 57, 'min_sum_hessian_in_leaf': 0.006296757603803257, 'feature_fraction': 0.8381244831959944, 'bagging_fraction': 0.7326320350377127, 'lambda_l1': 0.0223517282592696, 'lambda_l2': 0.265177341262348}. Best is trial 28 with value: 0.7048207896553889.


In [7]:
#探査の結果確認
trial = study.best_trial
print("acc(best)={:.4f}".format(trial.value))
display(trial.params)

acc(best)=0.7048


{'num_leaves': 233,
 'min_data_in_leaf': 49,
 'min_sum_hessian_in_leaf': 0.0068112646256021986,
 'feature_fraction': 0.8426315000433712,
 'bagging_fraction': 0.7317397959755858,
 'lambda_l1': 0.05156762599756539,
 'lambda_l2': 0.20885772437641695}

In [8]:
#ベストなハイパーパラメータの取得
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 233,
 'min_data_in_leaf': 49,
 'min_sum_hessian_in_leaf': 0.0068112646256021986,
 'feature_fraction': 0.8426315000433712,
 'bagging_fraction': 0.7317397959755858,
 'lambda_l1': 0.05156762599756539,
 'lambda_l2': 0.20885772437641695,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'learning_rate': 0.02,
 'n_estimators': 100000,
 'bagging_freq': 1,
 'seed': 123}

# LightGBM以外のモデル利用

In [9]:
#Titanicデータを用いた例：ロジスティクス回帰
# ファイル読み込み
df_train = pd.read_csv("../input/titanic/train.csv")

# データセット作成
x_train = df_train[["Pclass", "Age", "Embarked"]]
y_train = df_train[["Survived"]]

In [10]:
# 欠損値の確認
x_train.isnull().sum()

Pclass        0
Age         177
Embarked      2
dtype: int64

In [11]:
# 欠損値補間：数値データ 平均値補完
x_train["Age"] = x_train["Age"].fillna(x_train["Age"].mean())

# 欠損値補間：カテゴリ変数　最頻値補完
x_train["Embarked"] = x_train["Embarked"].fillna(x_train["Embarked"].mode()[0])

In [12]:
#カテゴリ変数の数値化（One-hot-encoding）
ohe = OneHotEncoder()
ohe.fit(x_train[["Embarked"]])
df_embarked = pd.DataFrame(
    ohe.transform(x_train[["Embarked"]]).toarray(), 
    columns=["Embarked_{}".format(col) for col in ohe.categories_[0]])

x_train = pd.concat([x_train, df_embarked], axis=1)
x_train = x_train.drop(columns=["Embarked"])

In [13]:
#数値データの正規化
x_train["Pclass"] = (x_train["Pclass"] -x_train["Pclass"].min()) / (x_train["Pclass"].max() - x_train["Pclass"].min()) 
x_train["Age"] = (x_train["Age"] -x_train["Age"].min()) / (x_train["Age"].max() - x_train["Age"].min()) 

In [14]:
#学習データと検証データの分割（ホールドアウト法）
x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=123)
print(x_tr.shape, x_va.shape, y_tr.shape, y_va.shape)

(712, 5) (179, 5) (712, 1) (179, 1)


In [15]:
#LogisticRegression ロジスティクス回帰
# モデル定義
from sklearn.linear_model import LogisticRegression
model_logis = LogisticRegression()

# 学習
model_logis.fit(x_tr, y_tr)

# 予測
y_va_pred = model_logis.predict(x_va)
print("accuracy:{:.4f}".format(accuracy_score(y_va, y_va_pred)))
print(y_va_pred[:5])

accuracy:0.7263
[0 1 0 1 0]


In [16]:
#確率値の取得
y_va_pred_proba = model_logis.predict_proba(x_va)
print(y_va_pred_proba[:5, :])

[[0.83621285 0.16378715]
 [0.23058311 0.76941689]
 [0.83244141 0.16755859]
 [0.32227072 0.67772928]
 [0.62569522 0.37430478]]


In [17]:
#Titanicデータを用いた例：SVM
# モデル定義
from sklearn.svm import SVC
model_svm = SVC(C=1.0, random_state=123, probability=True)#確率値の計算するパラメータをTrue

# 学習
model_svm.fit(x_tr, y_tr)

# 予測
y_va_pred = model_svm.predict(x_va)
print("accuracy:{:.4f}".format(accuracy_score(y_va, y_va_pred)))
print(y_va_pred[:5])

# 確率値の取得
y_va_pred_proba = model_svm.predict_proba(x_va)
print(y_va_pred_proba[:5, :])

accuracy:0.7151
[0 1 0 1 0]
[[0.73985924 0.26014076]
 [0.28242534 0.71757466]
 [0.73986177 0.26013823]
 [0.26828214 0.73171786]
 [0.58950192 0.41049808]]


# ニューラルネットワーク
### ニューラルネットワークの適用例：①全結合層のみのネットワークモデル

In [18]:
#Tensorflowインポート
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Embedding, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD

In [19]:
# tensorflowの再現性のためのシード指定
def seed_everything(seed):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

In [20]:
# ファイル読み込み
df_train = pd.read_csv("../input/titanic/train.csv")

# データセット作成
x_train = df_train[["Pclass", "Age", "Embarked"]]
y_train = df_train[["Survived"]]

In [21]:
# 欠損値補間 平均値補完
x_train["Age"] = x_train["Age"].fillna(x_train["Age"].mean())

# 正規化　補完後に0~1に間になるように正規化
for col in ["Pclass", "Age"]:
    value_min = x_train[col].min()
    value_max = x_train[col].max()
    x_train[col] = (x_train[col] - value_min) / (value_max - value_min)

In [22]:
# 欠損値補間　最頻値で補完
x_train["Embarked"] = x_train["Embarked"].fillna(x_train["Embarked"].mode()[0])

# one-hot-encodingで変換
ohe = OneHotEncoder()
ohe.fit(x_train[["Embarked"]])
df_embarked = pd.DataFrame(ohe.transform(x_train[["Embarked"]]).toarray(), 
                           columns=["Embarked_{}".format(col) for col in ohe.categories_[0]])
x_train = pd.concat([x_train.drop(columns=["Embarked"]), 
                     df_embarked], axis=1)

In [23]:
# 学習データと検証データの分割
x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=123)
print(x_tr.shape, x_va.shape, y_tr.shape, y_va.shape)

(712, 5) (179, 5) (712, 1) (179, 1)


In [24]:
#モデル定義　カラム数5:入力ノード5 隠れ層=3 10/10/5
def create_model():
    input_num = Input(shape=(5,))
    x_num = Dense(10, activation="relu")(input_num)
    x_num = BatchNormalization()(x_num)
    x_num = Dropout(0.3)(x_num)
    x_num = Dense(10, activation="relu")(x_num)
    x_num = BatchNormalization()(x_num)
    x_num = Dropout(0.2)(x_num)
    x_num = Dense(5, activation="relu")(x_num)
    x_num = BatchNormalization()(x_num)
    x_num = Dropout(0.1)(x_num)
    out = Dense(1, activation="sigmoid")(x_num)

    model = Model(inputs=input_num,
                  outputs=out,
                 )

    model.compile(
        optimizer="Adam",
        loss="binary_crossentropy",
        metrics=["binary_crossentropy"],
    )
    
    return model

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 dense (Dense)               (None, 10)                60        
                                                                 
 batch_normalization (BatchN  (None, 10)               40        
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 batch_normalization_1 (Batc  (None, 10)               40        
 hNormalization)                                             

In [25]:
#モデル学習
#ミニバッチサイズを8としてlossが5 回連続改善しない場合に学習率を1/10
#10回連続改善しなかった場合には強制終了
seed_everything(seed=123)
model = create_model()
model.fit(x=x_tr,
          y=y_tr,
          validation_data=(x_va, y_va),
          batch_size=8,
          epochs=10000,
          callbacks=[
              ModelCheckpoint(filepath="model_keras.h5", monitor="val_loss", mode="min", verbose=1, save_best_only=True, save_weights_only=True),
              EarlyStopping(monitor="val_loss", mode="min", min_delta=0, patience=10, verbose=1, restore_best_weights=True),
              ReduceLROnPlateau(monitor="val_loss", mode="min", factor=0.1, patience=5, verbose=1),
          ],
          verbose=1,
         )

Epoch 1/10000
72/89 [=======================>......] - ETA: 0s - loss: 0.7317 - binary_crossentropy: 0.7317
Epoch 1: val_loss improved from inf to 0.68175, saving model to model_keras.h5
89/89 [==============================] - 3s 7ms/step - loss: 0.7268 - binary_crossentropy: 0.7268 - val_loss: 0.6818 - val_binary_crossentropy: 0.6818 - lr: 0.0010
Epoch 2/10000
89/89 [==============================] - ETA: 0s - loss: 0.6753 - binary_crossentropy: 0.6753
Epoch 2: val_loss improved from 0.68175 to 0.66809, saving model to model_keras.h5
89/89 [==============================] - 0s 4ms/step - loss: 0.6753 - binary_crossentropy: 0.6753 - val_loss: 0.6681 - val_binary_crossentropy: 0.6681 - lr: 0.0010
Epoch 3/10000
87/89 [============================>.] - ETA: 0s - loss: 0.6838 - binary_crossentropy: 0.6838
Epoch 3: val_loss improved from 0.66809 to 0.65506, saving model to model_keras.h5
89/89 [==============================] - 0s 4ms/step - loss: 0.6801 - binary_crossentropy: 0.6801 - val

In [26]:
#モデルの評価
y_va_pred = model.predict(x_va, batch_size=8, verbose=1)
print("accuracy: {:.4f}".format(accuracy_score(y_va, np.where(y_va_pred>=0.5,1,0))))

23/23 [==============================] - 0s 1ms/step
accuracy: 0.7039


### ニューラルネットワークの適用例：②埋め込み層ありのネットワークモデル

In [27]:
# ファイル読み込み
df_train = pd.read_csv("../input/titanic/train.csv")

# データセット作成
x_train = df_train[["Pclass", "Age", "Cabin"]]
y_train = df_train[["Survived"]]